In [22]:
import sys
sys.path.append('../Python3Code')
from Chapter3.KalmanFilters import KalmanFilters
from Chapter3.DataTransformation import PrincipalComponentAnalysis, LowPassFilter
from Chapter3.ImputationMissingValues import ImputationMissingValues
from util.VisualizeDataset import VisualizeDataset
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

In [16]:
data = pd.read_csv('intermediate_datafiles/dataset.csv', index_col=0)

In [17]:
data.columns

Index(['barometer_X (hPa)', 'gyroscope_X (rad/s)', 'gyroscope_Y (rad/s)',
       'gyroscope_Z (rad/s)', 'accelerometer_X (m/s^2)',
       'accelerometer_Y (m/s^2)', 'accelerometer_Z (m/s^2)',
       'location_Velocity (m/s)', 'labelFreeStyle', 'labelBackStroke',
       'labelBreastStroke'],
      dtype='object')

In [18]:
# apply Kalman filter
kf = KalmanFilters()
data = kf.apply_kalman_filter(data, 'accelerometer_X (m/s^2)')
data = kf.apply_kalman_filter(data, 'accelerometer_Y (m/s^2)')
data = kf.apply_kalman_filter(data, 'accelerometer_Z (m/s^2)')
data = kf.apply_kalman_filter(data, 'gyroscope_X (rad/s)')
data = kf.apply_kalman_filter(data, 'gyroscope_Y (rad/s)')
data = kf.apply_kalman_filter(data, 'gyroscope_Z (rad/s)')

In [24]:
# apply imputation to barometer
imputator = ImputationMissingValues()
data = imputator.impute_interpolate(data, 'barometer_X (hPa)')

In [10]:
# plot uncleaned and cleaned data in same plot
plt.figure(figsize=(15, 5))

# indices on one baan
baan = slice(94500, 95500)

# Plotting the uncleaned data for each stroke
plt.plot(data['accelerometer_X (m/s^2)'][baan], label='Raw', color = 'blue')

# Plotting the cleaned data for each stroke
plt.plot(data['accelerometer_X (m/s^2)_kalman'][baan], label='Cleaned', color = 'red')

plt.ylabel('X (m/s^2)', fontsize=14)
plt.legend()
plt.show()

In [36]:
# apply low pass filter
lpf = LowPassFilter()
data = lpf.low_pass_filter(data, 'accelerometer_X (m/s^2)_kalman', 100, 10, order=20)
data = lpf.low_pass_filter(data, 'accelerometer_Y (m/s^2)_kalman', 100, 10, order=20)
data = lpf.low_pass_filter(data, 'accelerometer_Z (m/s^2)_kalman', 100, 10, order=20)
data = lpf.low_pass_filter(data, 'gyroscope_X (rad/s)_kalman', 100, 10, order=20)
data = lpf.low_pass_filter(data, 'gyroscope_Y (rad/s)_kalman', 100, 10, order=20)
data = lpf.low_pass_filter(data, 'gyroscope_Z (rad/s)_kalman', 100, 10, order=20)
data = lpf.low_pass_filter(data, 'barometer_X (hPa)', 100, 10, order=20)

In [11]:
# plot kalman filtered and low pass filtered data in same plot
plt.figure(figsize=(15, 5))
plt.plot(data['accelerometer_X (m/s^2)_kalman'][baan], label='Kalman filtered', color='blue')
plt.plot(data['accelerometer_X (m/s^2)_kalman_lowpass'][baan], label='Low pass filtered', color='red')
plt.ylabel('X (m/s^2)', fontsize=14)
plt.legend()
plt.show()

In [37]:
# apply PCA
pca = PrincipalComponentAnalysis()
data = pca.apply_pca(data, ['accelerometer_X (m/s^2)_kalman_lowpass', 
                            'accelerometer_Y (m/s^2)_kalman_lowpass', 
                            'accelerometer_Z (m/s^2)_kalman_lowpass', 
                            'gyroscope_X (rad/s)_kalman_lowpass', 
                            'gyroscope_Y (rad/s)_kalman_lowpass', 
                            'gyroscope_Z (rad/s)_kalman_lowpass',
                            'barometer_X (hPa)_lowpass',
                            ],
                            7)

In [13]:
pc_values = pca.determine_pc_explained_variance(data, ['accelerometer_X (m/s^2)_kalman_lowpass', 
                                                    'accelerometer_Y (m/s^2)_kalman_lowpass', 
                                                    'accelerometer_Z (m/s^2)_kalman_lowpass', 
                                                    'gyroscope_X (rad/s)_kalman_lowpass', 
                                                    'gyroscope_Y (rad/s)_kalman_lowpass', 
                                                    'gyroscope_Z (rad/s)_kalman_lowpass',
                                                    'barometer_X (hPa)_lowpass',
                                                    ]
                                                    )

plt.plot(range(1, 9), pc_values)
plt.xlabel('Principal component', fontsize=14)
plt.ylabel('Explained variance', fontsize=14)
plt.show()
                        

In [45]:
data.columns

Index(['barometer_X (hPa)', 'gyroscope_X (rad/s)', 'gyroscope_Y (rad/s)',
       'gyroscope_Z (rad/s)', 'accelerometer_X (m/s^2)',
       'accelerometer_Y (m/s^2)', 'accelerometer_Z (m/s^2)',
       'location_Velocity (m/s)', 'labelFreeStyle', 'labelBackStroke',
       'labelBreastStroke', 'accelerometer_X (m/s^2)_kalman',
       'accelerometer_Y (m/s^2)_kalman', 'accelerometer_Z (m/s^2)_kalman',
       'gyroscope_X (rad/s)_kalman', 'gyroscope_Y (rad/s)_kalman',
       'gyroscope_Z (rad/s)_kalman', 'accelerometer_X (m/s^2)_kalman_lowpass',
       'accelerometer_Y (m/s^2)_kalman_lowpass',
       'accelerometer_Z (m/s^2)_kalman_lowpass',
       'gyroscope_X (rad/s)_kalman_lowpass',
       'gyroscope_Y (rad/s)_kalman_lowpass',
       'gyroscope_Z (rad/s)_kalman_lowpass',
       'barometer_X (hPa)_kalman_lowpass',
       'location_Velocity (m/s)_kalman_lowpass', 'pca_1', 'pca_2', 'pca_3',
       'pca_4', 'pca_5', 'pca_6', 'pca_7', 'barometer_X (hPa)_lowpass'],
      dtype='object')

In [46]:
data = data[['accelerometer_X (m/s^2)_kalman_lowpass',
            'accelerometer_Y (m/s^2)_kalman_lowpass',
            'accelerometer_Z (m/s^2)_kalman_lowpass',
            'gyroscope_X (rad/s)_kalman_lowpass',
            'gyroscope_Y (rad/s)_kalman_lowpass',
            'gyroscope_Z (rad/s)_kalman_lowpass',
            'barometer_X (hPa)_lowpass',
            'labelFreeStyle',
            'labelBackStroke',
            'labelBreastStroke',
            'pca_1',
            'pca_2',
            'pca_3']]

In [51]:
# clean up names
data = data.rename(columns={'accelerometer_X (m/s^2)_kalman_lowpass': 'accelerometer_X (m/s^2)',
                    'accelerometer_Y (m/s^2)_kalman_lowpass': 'accelerometer_Y (m/s^2)',
                    'accelerometer_Z (m/s^2)_kalman_lowpass': 'accelerometer_Z (m/s^2)',
                    'gyroscope_X (rad/s)_kalman_lowpass': 'gyroscope_X (rad/s)',
                    'gyroscope_Y (rad/s)_kalman_lowpass': 'gyroscope_Y (rad/s)',
                    'gyroscope_Z (rad/s)_kalman_lowpass': 'gyroscope_Z (rad/s)',
                    'barometer_X (hPa)_lowpass': 'barometer_X (hPa)'})

In [52]:
data.columns

Index(['accelerometer_X (m/s^2)', 'accelerometer_Y (m/s^2)',
       'accelerometer_Z (m/s^2)', 'gyroscope_X (rad/s)', 'gyroscope_Y (rad/s)',
       'gyroscope_Z (rad/s)', 'barometer_X (hPa)', 'labelFreeStyle',
       'labelBackStroke', 'labelBreastStroke', 'pca_1', 'pca_2', 'pca_3',
       'pca_4', 'pca_5', 'pca_6', 'pca_7'],
      dtype='object')

In [56]:
# export to csv
data.to_csv('intermediate_datafiles/cleaned_dataset.csv')